<a href="https://colab.research.google.com/github/shahtvisha/TextSummarizationUsingDeepLearning/blob/main/FineTunedModel%20_BioT5v1_TextSummarizationEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0edabd50dc0fb09a2fb9335c1bea81b6061e2dd204229a645ff7889740c8482e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are install

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_metric
import torch

# Load ROUGE metric
rouge = load_metric("rouge")


<ipython-input-3-f1157f1c0144>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [4]:
import pandas as pd
csv_file_path = "/content/preprocessed_pubmed_dataset.xls"
pubmed_data = pd.read_csv(csv_file_path)
pubmed_data['article'] = pubmed_data['article'].astype(str)
pubmed_data['abstract'] = pubmed_data['abstract'].astype(str)
pubmed_data.dropna()
pubmed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27869 entries, 0 to 27868
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   article   27869 non-null  object
 1   abstract  27869 non-null  object
dtypes: object(2)
memory usage: 435.6+ KB


In [5]:
pubmed_data['article'] = pubmed_data['article'].apply(lambda x: str(x))
pubmed_data['abstract'] = pubmed_data['abstract'].apply(lambda x: str(x))
pubmed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27869 entries, 0 to 27868
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   article   27869 non-null  object
 1   abstract  27869 non-null  object
dtypes: object(2)
memory usage: 435.6+ KB


In [6]:
model_name = "TvishaShah/BioT5"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [7]:
def summarize(text, model, tokenizer, max_input_length=512, max_output_length=150):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=max_input_length, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_output_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [9]:
batch_size = 1
max_iterations = 50

# Prepare to calculate ROUGE scores
rouge = load_metric("rouge")
rouge_scores_list = []

# Iterate over the dataset with a limit of 50 iterations
for i in range(max_iterations):
    row = pubmed_data.iloc[i]
    generated_summary = summarize(row['article'], model, tokenizer)
    rouge_scores = rouge.compute(predictions=[generated_summary], references=[row['abstract']])
    rouge_scores_list.append(rouge_scores)
    print(f"Processed {i + 1}/{max_iterations}")

# Calculate average ROUGE scores over all batches
average_rouge_scores = {
    "rouge1": sum(score["rouge1"].mid.fmeasure for score in rouge_scores_list) / len(rouge_scores_list),
    "rouge2": sum(score["rouge2"].mid.fmeasure for score in rouge_scores_list) / len(rouge_scores_list),
    "rougeL": sum(score["rougeL"].mid.fmeasure for score in rouge_scores_list) / len(rouge_scores_list)
}

Processed 1/50
Processed 2/50
Processed 3/50
Processed 4/50
Processed 5/50
Processed 6/50
Processed 7/50
Processed 8/50
Processed 9/50
Processed 10/50
Processed 11/50
Processed 12/50
Processed 13/50
Processed 14/50
Processed 15/50
Processed 16/50
Processed 17/50
Processed 18/50
Processed 19/50
Processed 20/50
Processed 21/50
Processed 22/50
Processed 23/50
Processed 24/50
Processed 25/50
Processed 26/50
Processed 27/50
Processed 28/50
Processed 29/50
Processed 30/50
Processed 31/50
Processed 32/50
Processed 33/50
Processed 34/50
Processed 35/50
Processed 36/50
Processed 37/50
Processed 38/50
Processed 39/50
Processed 40/50
Processed 41/50
Processed 42/50
Processed 43/50
Processed 44/50
Processed 45/50
Processed 46/50
Processed 47/50
Processed 48/50
Processed 49/50
Processed 50/50


In [10]:
print(f"Average ROUGE Scores after {max_iterations} iterations:")
print(f"ROUGE-1: {average_rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {average_rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {average_rouge_scores['rougeL']:.4f}")

Average ROUGE Scores after 50 iterations:
ROUGE-1: 0.2928
ROUGE-2: 0.1132
ROUGE-L: 0.1923
